In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService


create pages list for arabam.com to get each car's link

In [ ]:
def get_pages_list():
    first_page = "https://www.arabam.com/ikinci-el/otomobil?take=50"
    page_list = []
    for i in range(2, 50):
        page_list.append(first_page + "&page=" + str(i))
    print(page_list)


In [ ]:
get_pages_list()

In [ ]:
def scroll_down():
    for j in range(2):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")  # Kaydırma İşlemi
        time.sleep(1)  # wait for one second after each scroll


In [4]:
chrome_driver_path = "/Users/zehragul/Documents/Development/chromedriver"
service = ChromeService(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
with open("car_list.txt","r") as data_file:
        cars = data_file.readlines()


In [ ]:
result = []

for car in cars:

    driver.get(cars)
    driver.implicitly_wait(2)
    scroll_down()


    try:
        fiyat = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/div[1]/div/div').text.replace("TL","")
        fiyat = fiyat.replace(".", "")
    except:
        fiyat = np.nan
    
    try:
        yil = int(driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[6]/span[2]').text)
    except:
        yil = np.nan
    
    try:
        marka = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[3]/span[2]').text
    except:
        marka = np.nan
    
    try:
        seri = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[4]/span[2]').text
    except:
        seri = np.nan
    
    try:
        model = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[5]/span[2]').text
    except:
        model = np.nan

    try:
        kilometre = float(driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[7]/span[2]').text.replace("km","").strip())
    except:
        kilometre = np.nan
    
    try:
        vites_tipi = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[8]/span[2]').text
    except:
        vites_tipi = np.nan
    
    try:
        yakit_tipi = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[9]/span[2]').text
    except:
        yakit_tipi = np.nan
    
    try:
        motor_hacmi = float(driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[11]/span[2]').text.replace("cc",""))
    except:
        motor_hacmi = np.nan
    
    try:
        motor_gucu = float(driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[12]/span[2]').text.replace("hp",""))
    except:
        motor_gucu = np.nan

    try:
        cekis = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[13]/span[2]').text
    except:
        cekis = np.nan
    
    try:
        ort_yakit_tuk = float(driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[14]/span[2]').text.replace("lt",""))
    except:
        ort_yakit_tuk = np.nan
    
    try:
        yakit_deposu = float(driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[15]/span[2]').text.replace("lt",""))
    except:
        yakit_deposu = np.nan
    
    try:
        boya_degisen = driver.find_element(by=By.XPATH, value='//*[@id="js-hook-for-observer-detail"]/div[2]/ul/li[16]/a').text
    except:
        boya_degisen = np.nan
    
    new_row =[fiyat,yil,renk, marka , seri , model, kilometre, vites_tipi, yakit_tipi, cekis, ort_yakit_tuk, yakit_deposu,tork, motor_hacmi, motor_gucu, koltuk_sayisi, boya_degisen]
    result.append(new_row)

    
columns = ["fiyat", "yil", "marka" , "seri" , "model", "kilometre", "vites_tipi", "yakit_tipi", "cekis", "ort_yakit_tuk", "yakit_deposu", "motor_hacmi", "motor_gucu", "boya_degisen"]
df = pd.DataFrame.from_records(result, columns=columns)
df.to_csv("cars.csv", index_label=False)
print(f"link sayisi :  {len(result)}")